<div style="display: flex; align-items: center;">
    <img src="https://github.com/nagelt/Teaching_Scripts/raw/9d9e29ecca4b04eaf7397938eacbf116d37ddc93/Images/TUBAF_Logo_blau.png" width="500" height="auto" height="auto" style="margin-right: 100px;" />
    <div>
        <p><strong>Prof. Dr. Thomas Nagel</strong></p>
        <p>Chair of Soil Mechanics and Foundation Engineering<br>Geotechnical Institute<br>Technische Universität Bergakademie Freiberg.</p>
        <p><a href="https://tu-freiberg.de/en/soilmechanics">https://tu-freiberg.de/en/soilmechanics</a></p>
    </div>
</div>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from pypardiso import spsolve

#Some plot settings
%run plot_functions/plot_settings.py

# Quick tests on nodal fluxes

## Divergence-free flow

Consider a linear concentration gradient of $\nabla c = -1$ on $x \in [0,1]$ for $v = 1$ and $D = 1$. We calculate nodal contributions based on the following terms (no units for the moment):

\begin{align}
    f_1 &= \int \limits_0^1 \nabla N v c \text{d}x
    \
    f_2 &= \int \limits_0^1 N v \nabla c \text{d}x
    \
    f_3 &= -\int \limits_0^1 \nabla N v \nabla c \text{d}x
\end{align}

$f_1$ is a treatment used for advection in total flux models, $f_2$ is motivated by the implementation of advection with substituted mass balance, $f_3$ for diffusion.

## Finite elements in 1D

We first create an element class. An element knows the number of nodes it has, their IDs in the global node vector, and the coordinates of its nodes. Linear elements have 2 nodes and 2 quadrature points, quadratic elements 3 nodes and 3 quadrature points. The natural coordinates of the element run from -1 to 1, and the quadrature points and weights are directly taken from Numpy.

In [8]:
#element class
class line_element():#local coordinates go from -1 to 1
    #takes number of nodes, global nodal coordinates, global node ids
    def __init__(self, nnodes=2, ncoords=[0.,1.], nids=[0,1]):
        self.__nnodes = nnodes
        if (len(ncoords) != self.__nnodes):
            raise Exception("Number of coordinates does not match number \
                            of nodes of element (%i vs of %i)" %(self.__nnodes,len(ncoords)))
        else:
            self.__coords = np.array(ncoords)
        
        self.__natural_coords = (self.__coords-self.__coords[0])/(self.__coords[-1]-self.__coords[0])*2. - 1.
        
        if (len(nids) != self.__nnodes):
            raise Exception("Number of node IDs does not match number \
                            of nodes of element (%i vs of %i)" %(self.__nnodes,len(nids)))
        else:
            self.__global_ids = np.array(nids)
        self.__quad_degree = self.__nnodes
        self.__quad_points, self.__quad_weights = np.polynomial.legendre.leggauss(self.__quad_degree)
                

In [9]:
#N
def shape_function(element_order,xi):
    if (element_order == 1):
            return np.array([1])
    elif (element_order == 2): #-1,1
            return np.array([(1.-xi)/2., (1.+xi)/2.])
    elif (element_order == 3): #-1, 0, 1
            return np.array([(xi - 1.)*xi/2., (1-xi)*(1+xi), (1+xi)*xi/2.])
    elif (element_order == 4): #-1, -1/3, 1/3, 1
            return np.array([9/16*(1-xi)*(xi**2 - 1/9),
                            27/16*(xi**2-1)*(xi-1/3),
                            27/16*(1-xi**2)*(xi+1/3),
                            9/16*(xi+1)*(xi**2-1/9)])
        
#dN_dxi
def dshape_function_dxi(element_order,xi):
    if (element_order == 1):
            return np.array([0])
    elif (element_order == 2): #-1,1
        return np.array([-0.5, 0.5])  #xi only later for plotting dimensions
    elif (element_order == 3):#-1,0,1
        return np.array([xi - 0.5,-2.*xi,xi + 0.5])
    elif (element_order == 4): #-1, -1/3, 1/3, 1
            return np.array([-27*xi**2/16 + 9*xi/8 + 1/16,
                            81/16*xi**2 - 9/8 * xi - 27/16, 
                            -81/16*xi**2 - 9/8 * xi +27/16,
                            27*xi**2/16 + 9*xi/8 - 1/16])

#dz_dxi
def element_jacobian(element,xi):
    element_order = element._line_element__nnodes
    Jacobian = 0.
    Jacobian += dshape_function_dxi(element_order,xi).dot(element._line_element__coords)
    return Jacobian

#dN_dz
def grad_shape_function(element,xi):
    element_order = element._line_element__nnodes
    Jac = element_jacobian(element,xi)
    return dshape_function_dxi(element_order,xi)/Jac

## Local assember

In [99]:
def test_assembler(elem, c_nodes, v_nodes):
    element_order = elem._line_element__nnodes
    b_1 = np.zeros(element_order)
    b_2 = np.zeros(element_order)
    b_3 = np.zeros(element_order)
    b_4 = np.zeros(element_order)
    #z_nodes = elem._line_element__coords
    for i in range(elem._line_element__quad_degree):
        #local integration point coordinate
        xi = elem._line_element__quad_points[i]
        #shape function
        N = shape_function(element_order,xi)
        #gradient of shape function
        dN_dX = grad_shape_function(elem,xi)
        #determinant of Jacobian
        detJ = np.abs(element_jacobian(elem,xi))
        #integration weight
        w = elem._line_element__quad_weights[i]
        
        #global integration point coordinate (for spatially varying properties)
        c_glob = np.dot(N,c_nodes)
        v_glob = np.dot(N,v_nodes)
        grad_c = np.dot(dN_dX,c_nodes)
        grad_v = np.dot(dN_dX,v_nodes)
        #evaluation of local material/structural properties
        #assembly of local RHS
        b_1 += dN_dX * c_glob * v_glob * w * detJ
        b_2 += N * grad_c * v_glob * w * detJ
        b_3 -= dN_dX * grad_c * w * detJ
        b_4 += N * c_glob * grad_v * w * detJ
    return b_1, b_2, b_3, b_4

In [103]:
#generate elements of order 1 to 5 and calculate fluxes
for i in range(1,4):
    nodes = np.linspace(0,i,i+1)
    ncoords = np.linspace(0,1,i+1)
    elem = line_element(i+1,ncoords,nodes)
    nconc = np.flip(ncoords)
    f1, f2, f3, f4 = test_assembler(elem,nconc,np.ones(len(nodes)))
    print("element order: ", i)
    print("f_1: ", f1)
    print("f_2: ", f2)
    print("f_1 + f_2: ", f1+f2)
    print("f_3: ", f3)
    print("---\n")

element order:  1
f_1:  [-0.5  0.5]
f_2:  [-0.5 -0.5]
f_1 + f_2:  [-1.  0.]
f_3:  [-1.  1.]
---

element order:  2
f_1:  [-0.83333333  0.66666667  0.16666667]
f_2:  [-0.16666667 -0.66666667 -0.16666667]
f_1 + f_2:  [-1.00000000e+00  2.22044605e-16 -1.11022302e-16]
f_3:  [-1.  0.  1.]
---

element order:  3
f_1:  [-0.875  0.375  0.375  0.125]
f_2:  [-0.125 -0.375 -0.375 -0.125]
f_1 + f_2:  [-1.00000000e+00 -3.88578059e-16  4.44089210e-16 -1.38777878e-17]
f_3:  [-1.00000000e+00 -7.77156117e-16  8.88178420e-16  1.00000000e+00]
---



Diffusion is always correctly quantified by $f_3$: 1 influx, 1 outflux, no internal fluxes in the domain.

Advection is approached (1 influx and 0 outflux) by $f_1$ as the element order increases, but some averaging remains also at higher order. Internal fluxes show the redistribution of mass from left to right.

The sum $f_1 + f_2$ recovers the advective flux with expected values on the boundaries, without internal redistribution information.

## Divergent flux

Now we add a linear velocity profile with mean value of 1 and repeat the test.

In [109]:
ncoords = np.linspace(0,1,2)
v = np.ones(len(ncoords))-(ncoords-0.5)*0.5
v

array([1.25, 0.75])

In [105]:
#generate elements of order 1 to 5 and calculate fluxes
for i in range(1,4):
    nodes = np.linspace(0,i,i+1)
    ncoords = np.linspace(0,1,i+1)
    elem = line_element(i+1,ncoords,nodes)
    nconc = np.flip(ncoords)
    f1, f2, f3, f4 = test_assembler(elem,nconc,np.ones(len(nodes))-(ncoords-0.5)*0.5)
    print("element order: ", i)
    print("f_1: ", f1)
    print("f_2: ", f2)
    print("f_1 + f_2: ", f1+f2)
    print("f_1 + f_2 + f_4: ", f1+f2+f4)
    print("f_3: ", f3)
    print("---\n")

element order:  1
f_1:  [-0.54166667  0.54166667]
f_2:  [-0.54166667 -0.45833333]
f_1 + f_2:  [-1.08333333  0.08333333]
f_1 + f_2 + f_4:  [-1.25000000e+00  1.38777878e-16]
f_3:  [-1.  1.]
---

element order:  2
f_1:  [-0.95833333  0.83333333  0.125     ]
f_2:  [-0.20833333 -0.66666667 -0.125     ]
f_1 + f_2:  [-1.16666667e+00  1.66666667e-01 -1.24900090e-16]
f_1 + f_2 + f_4:  [-1.25000000e+00  2.77555756e-16 -1.24900090e-16]
f_3:  [-1.  0.  1.]
---

element order:  3
f_1:  [-1.04791667  0.58125     0.35625     0.11041667]
f_2:  [-0.14791667 -0.43125    -0.31875    -0.10208333]
f_1 + f_2:  [-1.19583333  0.15        0.0375      0.00833333]
f_1 + f_2 + f_4:  [-1.25000000e+00 -1.94289029e-16  2.22044605e-16 -3.98986399e-17]
f_3:  [-1.00000000e+00 -7.77156117e-16  8.88178420e-16  1.00000000e+00]
---



Der outflux von 0 wird in Ordnung 2 erreicht, in 3 aber wieder verfehlt (dennoch eine Ordnung besser als im linearen Element. Im linearen Element hätten wir allerdings konstante Geschwindigkeit, wenn wir mit einem hydraulischen Potenzial an den Knoten arbeiten.). Der Influx von 1.25 wird zunehmend besser approximiert, aber Unterschiede bleiben. Wenn man den Term verfollständigt um

$$
    f_4 = \int \limits_{0}^1 N c \nabla v \text{d}x
$$

geht die Bilanz wieder auf. Im Standard FE allerdings nicht so easy.